# Importing and file opening

In [36]:
import pyarrow.parquet as pq
import numpy as np
import awkward as ak
import uproot
# import particle
import vector
vector.register_awkward()

fname = "/nfs/dust/cms/user/matsch/for_Artak/0520A050-AF68-EF43-AA5B-5AA77C74ED73.root"

fname_HH = "/nfs/dust/cms/user/frahmmat/tutorial_files/HHtobbVV_4B83EAAD-1DEF-1641-BF6E-E9D72832F33A.root"
fname_tt = "/nfs/dust/cms/user/frahmmat/tutorial_files/tt_sl_0520A050-AF68-EF43-AA5B-5AA77C74ED73.root"

file = uproot.open(fname)
tree = file["Events"]
print(tree)
# print(tree.keys())  # 1628 keys

<TTree 'Events' (1628 branches) at 0x2b030b5961d0>


# directly accessing fields via filter

In [37]:
tree = file["Events"]

fields = ["pt", "eta", "phi", "mass", "charge", "btagDeepB"]

electrons = tree.arrays(filter_name=["Electron_" + f for f in fields], entry_stop=100_000)
muons = tree.arrays(filter_name=["Muon_" + f for f in fields], entry_stop=100_000)
jets = tree.arrays(filter_name=["Jet_" + f for f in fields], entry_stop=100_000)

In [38]:
# switch naming of fields from "{Object}_{field}" to "{field}"
electrons = ak.zip({key.replace("Electron_",""):electrons[key] for key in electrons.fields}, with_name="Momentum4D")
muons = ak.zip({key.replace("Muon_",""):muons[key] for key in muons.fields}, with_name="Momentum4D")
jets = ak.zip({key.replace("Jet_",""):jets[key] for key in jets.fields}, with_name="Momentum4D")

# combine fields into a single awkward array via ak.zip
events = ak.zip({"Electron": electrons, "Muon": muons, "Jet": jets}, depth_limit=1)

# save output as parquet file
f_out_name = "tt_sl_test.parquet"
ak.to_parquet(events, f_out_name)

Muon fields: ['eta', 'mass', 'phi', 'pt', 'charge']
Jet fields: ['btagDeepB', 'eta', 'mass', 'phi', 'pt']
<class 'vector._backends.awkward_.MomentumArray4D'>
None
<class 'vector._backends.awkward_.MomentumArray4D'>
event fields: ['Electron', 'Muon', 'Jet']
None


# opening parquet with awkward

In [42]:
events_new = ak.from_parquet("tt_sl_test.parquet")

print(type(events_new.Electron))
# enable 4-vector behavior (pt,eta,phi,mass)
behaviors = {
    "Jet": "Momentum4D",
    "Electron": "Momentum4D",
    "Muon": "Momentum4D",
}
for f in events_new.fields:
    if f in behaviors:
        events_new[f] = ak.with_name(events_new[f], behaviors[f])
    
print(events_new.fields)
print(type(events_new.Electron))

<class 'awkward.highlevel.Array'>
['Electron', 'Muon', 'Jet']
<class 'vector._backends.awkward_.MomentumArray4D'>


In [43]:
# playing around with fields, testing vector behaviour
jets = events_new.Jet

print(ak.num(jets))

print(jets.mass)
jets = jets[ak.num(jets)>1]
print((jets[:,0]+jets[:,1]).mass)
print(jets.pt)
print((jets[:,0]+jets[:,1]).pt)
print(jets.E)
print(np.sqrt(jets.pt**2 + jets.pz**2 + jets.mass**2))


[6, 7, 7, 9, 7, 8, 6, 5, 6, 8, 6, 10, 9, ... 6, 11, 7, 7, 8, 11, 8, 5, 9, 16, 11, 8]
[[11.3, 9.84, 7.24, 4.14, 4.92, 3.79], ... 8.74, 5.56, 5.16, 6.71, 4.8, 6.57, 4.31]]
[86.9, 303, 203, 542, 108, 246, 105, 309, ... 271, 149, 184, 412, 59.7, 327, 147]
[[101, 62.8, 47.9, 22.3, 19.2, 15.5], ... 42.2, 40.2, 35.8, 33.5, 27, 21.2, 15.1]]
[157, 56.7, 146, 84.8, 108, 83.2, 35.4, ... 78.4, 76, 37.1, 67.9, 194, 23.3, 38.2]
[[118, 66.1, 49.1, 274, 49.1, 30.9], ... 43.2, 79.3, 48.1, 35.8, 182, 156, 105]]
[[118, 66.1, 49.1, 274, 49.1, 30.9], ... 43.2, 79.3, 48.1, 35.8, 182, 156, 105]]


# alternatively: accessing fields via coffea

In [78]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema
from coffea.nanoevents.methods import candidate

events = NanoEventsFactory.from_root(
    file,
    entry_stop=100000,
    metadata={"dataset": "tt_sl"},
    schemaclass=BaseSchema,
).events()

muons = ak.zip(
    {
        "pt": events.Muon_pt,
        "eta": events.Muon_eta,
        "phi": events.Muon_phi,
        "mass": events.Muon_mass,
        "charge": events.Muon_charge,
    },
    with_name="PtEtaPhiMCandidate",
    behavior=candidate.behavior,
)
jets = ak.zip(
    {
        "pt": events.Muon_pt,
        "eta": events.Muon_eta,
        "phi": events.Muon_phi,
        "mass": events.Muon_mass,
        "charge": events.Muon_charge,
    },
    with_name="PtEtaPhiMCandidate",
    behavior=candidate.behavior,
)